In [1]:
import pandas as pd
import numpy as np

In [2]:
ratings = pd.read_csv('/home/sjkim/recommendSystem/finalproject/Amazon_ratings.csv', encoding='latin-1')
ratings.head()

,item_id,user_id,rating,timestamp
0,A0955928C2RRWOWZN7UC,B00191WVF6,4.0,1487289600
1,A0955928C2RRWOWZN7UC,B005WY3TMA,4.0,1434240000
2,A0955928C2RRWOWZN7UC,B0090XWU8S,4.0,1492819200
3,A0955928C2RRWOWZN7UC,B00FXYTLIK,4.0,1436486400
4,A0955928C2RRWOWZN7UC,B00HMZG3YS,4.0,1436486400


In [3]:
len(ratings)

99742

In [8]:
items = pd.read_csv('/home/sjkim/recommendSystem/finalproject/Amazon_ratings.csv', encoding='latin-1')
items = items.set_index('item_id')
items.head(2)

,user_id,rating,timestamp
item_id,,,
A0955928C2RRWOWZN7UC,B00191WVF6,4.0,1487289600
A0955928C2RRWOWZN7UC,B005WY3TMA,4.0,1434240000


In [10]:
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id'] # 기존 데이터에서 각 user_id별로 train_set과 test_set의 비율을 동일하게 유지하기 위해서 user_id를 기준으로 stratified sampling 방식을 사용하기 위해서
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y, random_state=12) # stratify=y 있으면 오류나서 제거


In [11]:
# ratings에 있는 평가 데이터를 아이템별 평점 평균을 구한 후 item_mean 변수에 저장
item_mean = ratings.groupby(['item_id'])['rating'].mean()
item_mean

item_id
A0955928C2RRWOWZN7UC    4.000000
A100UD67AHFODS          4.966667
A100WO06OQR8BQ          4.153846
A1013Q9SD2KIE1          3.692308
A1018D502E27E6          3.727273
                          ...   
AZX2RDN9YXZAE           3.416667
AZXFS8GCTSQ5R           4.055556
AZXJJM4F1SY3            4.846154
AZYWYG2YYXH7O           5.000000
AZZYW4YOE1B6E           4.461538
Name: rating, Length: 6368, dtype: float64

In [12]:
item_sort = item_mean.sort_values(ascending=False)[:5]
item_sort

item_id
A5ZU5Y9O8MULS     5.0
A1P646XPZGA4BL    5.0
A3J18C4T2JWI2F    5.0
A2RBO3RBHPZBAN    5.0
A1QQ4U4RTI0A1M    5.0
Name: rating, dtype: float64

In [13]:
# best-seller 추천
def recom_item(n_items):
    item_sort = item_mean.sort_values(ascending=False)[:n_items] # movie_mean을 내림차순으로 정렬하고 상위부터 지정된 개수(n_item)만큼 item_sort에 저장
    return list(item_sort.index)

In [14]:
recom_item(5)

['A5ZU5Y9O8MULS',
 'A1P646XPZGA4BL',
 'A3J18C4T2JWI2F',
 'A2RBO3RBHPZBAN',
 'A1QQ4U4RTI0A1M']

In [15]:
# RMSE 계산
def RMSE(y_true, y_pred):
    import numpy as np
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

rmse = []                                   # 결과를 저장할 rmse 변수 초기화
for user in set(ratings.index):             # 겹치지 않은 unique한 게 뽑힘
    y_true = ratings.loc[user]['rating']    # 각 사용자가 평가한 모든 아이템의 평점을 y_true에 저장
    y_pred = item_mean[ratings.loc[user]['item_id']]    # 해당 사용자가 평가한 아이템의 평균평점을 y_pred에 저장
    accuracy = RMSE(y_true, y_pred)         # 실제 rating과 평균 rating의 RMSE 계산
    rmse.append(accuracy)
print(np.mean(rmse))

0.5854778893653736
